# **Practical session - Deep Learning for object detection**
**Master 2 GeoData Science**

*2021-2022*

## **1/ Introduction**
The purpose of this lab is to experiment the one-stage YOLO framework for object detection. We will learn how to train YOLO on Colab using the Darknet repository (https://github.com/AlexeyAB/darknet). Then we will apply the latest versions (YOLOv3 and YOLOv4) for vehicle detection from aerial images using the **MUNICH dataset**. If your Colab GPU is cool enough, we could perform the vehicle detection on larger datasets as VEDAI (aerial) or XVIEW (satellite).

Before starting, let's prepare your Colab notebook
- check the cuda version
- enable GPU (`Runtime > change runtime type`
 ) and check its configuration
- mount your drive to allow Colab to acess to your Google Drive (for e.g., reading/writing images)

In [1]:
# check the CUDA version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Wed_Jun__2_19:25:35_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.4, V11.4.48
Build cuda_11.4.r11.4/compiler.30033411_0


In [2]:
# After enabling the GPU, check its configuration
!nvidia-smi

Mon Dec 13 09:44:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.11       Driver Version: 471.11       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8     6W /  N/A |    356MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **2/ Cloning and Building Darknet from Cloud**

Let's first clone darknet from the AlexeyAB's repository, then adjust the Makefile to enable OPENCV and GPU for darknet and finally compile (build) darknet.

In [3]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
Updating files:  77% (1588/2050)
Updating files:  78% (1599/2050)
Updating files:  79% (1620/2050)
Updating files:  80% (1640/2050)
Updating files:  81% (1661/2050)
Updating files:  82% (1681/2050)
Updating files:  83% (1702/2050)
Updating files:  84% (1722/2050)
Updating files:  85% (1743/2050)
Updating files:  86% (1763/2050)
Updating files:  87% (1784/2050)
Updating files:  88% (1804/2050)
Updating files:  89% (1825/2050)
Updating files:  90% (1845/2050)
Updating files:  91% (1866/2050)
Updating files:  92% (1886/2050)
Updating files:  93% (1907/2050)
Updating files:  94% (1927/2050)
Updating files:  95% (1948/2050)
Updating files:  96% (1968/2050)
Updating files:  97% (1989/2050)
Updating files:  98% (2009/2050)
Updating files:  99% (2030/2050)
Updating files: 100% (2050/2050)
Updating files: 100% (2050/2050), done.


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

c:\Users\rufai\Downloads\darknet


'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# check the current path and list
!pwd
!ls

In [ ]:
# compile (build) darknet using the make command
# --> so that you can then use the darknet executable file to predict or to train object detectors
# Do not worry about any warnings

!make

In [ ]:
# check if the darknet executable file is fine

!./darknet

## **3/ Download pre-trained YOLO weights**
Several YOLO models have been trained already on the COCO dataset which has 80 classes that it can predict. 

We will now download the pretrained weights of YOLOv3 and YOLOv4 so that we can run these two models on these pretrained classes and get detections of any image.

In [ ]:
# get YOLOv3 pre-trained weights
!wget https://pjreddie.com/media/files/yolov3.weights


In [ ]:
# get YOLOv4 pre-trained weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

## **4/ Define some helper functions**
These three functions are helper functions that will allow you to show the image in your Colab Notebook after running your detections, as well as upload and download images to and from your Cloud Google virtual machine.

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(2*width, 2*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(12, 8)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

## **5/ Run your detections with Darknet using YOLO models**
Darknet is now built and ready to run detections using YOLO models in the cloud! You can find out which sorts of classes the pre-trained YOLO weights can detect by clicking here. [COCO CLASSES](http://cocodataset.org/#explore)

The object detector can be run using the following command
```bash
!./darknet detector test <path to .data file> <path to config> <path to weights> <path to image>
```
Darknet comes with a few images already installed in the darknet/data/ folder.

**Note:** After running detections OpenCV can't open the image instantly in the cloud so we must run: 
```bash
imShow('predictions.jpg')
```
This will output the image with the detections shown. The most recent detections are always saved to 'predictions.jpg'

Try out the examples below for yourself!

In [ ]:
imShow('data/person.jpg')

In [ ]:
# run darknet detection on test images using YOLOv3
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

In [ ]:
# show the detection result
imShow('predictions.jpg')

In [ ]:
# do the same thing using YOLOv4
# ..............
# .............


In [ ]:
# then test on other images from the data folder
# ..............
# .............

## **6/ Uploading image files from your local machine or your Google Drive for detection**

You may want to run detections on more than just the images within the darknet/data/ folder. This step will show you how to upload local or Google Drive files to the cloud VM and run detections on them!

### a) From Local Files
To upload local files just use our helper function by running 'upload()' as seen below. Click Choose Files and select the image from your local machine that you want to upload to the cloud VM.

Otherwise, you can simple use the Upload button in the File Explorer on the left side of your notebook.

To detect objects from your image using YOLOv3, simple do.

```
!./darknet detector test cfg/coco.data cfg/yolov3.cfg yolov3.weights ../<your image name>
```

In [ ]:
!pwd

In [ ]:
upload()

In [ ]:
imShow('img_test_02.jpg')

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights img_test_02.jpg

imShow('predictions.jpg')

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg

imShow('predictions.jpg')

In [ ]:
# detect with YOLOv3 or YOLOv4
# ..............
# .............

# images.txt

# /data/images1.jpg
# /data/images2.jpg

### b) From your Google Drive by mounting your drive

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# cd back into the darknet folder to run detections
%cd darknet

In [ ]:
# then upload your images and test on them
#................

### c) Download Files to Local Machine or Google Drive from Cloud VM

You can also easily download images from your cloud VM to save to your local machine or Google Drive.

In [ ]:
# LOCAL MACHINE DOWNLOAD
# using helper function to download the prediction image

download('predictions.jpg')

In [ ]:
# GOOGLE DRIVE DOWNLOAD
# note that I can change what the image name is saved as (I am saving it as detection1.jpg)

!cp predictions.jpg /mydrive/images/detection1.jpg

## **7/ Let's try with a video**

You can also run YOLOv4 object detections on video in your Google Colab VM (from your local machine or from your Google Drive).

In [ ]:
# check
!pwd


In [ ]:
# upload the video of your choosing!
upload()


In [ ]:
# try this
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show test_video.mp4 -i 0 -out_filename results_video.avi


In [ ]:
# download the video with detections shown
download('results_video.avi')


## **8/ Now, customize YOLO prediction with the different thresholds**

There is a flag '-thresh' you can use to add a threshold for confidences on the detections.  Only detections with a confidence level above the threshold you set will be returned.

In the example below we run darknet with YOLOv4 without a threshold on the test image dog.jpg, without and with the '-thresh' flag. Let's observe the result.


In [ ]:
# this is ran without the threshold flag set
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg
imShow('predictions.jpg')

In [ ]:
# same detections but ran with the threshold flag set to 0.5 
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg -thresh 0.5
imShow('predictions.jpg')


You can also output bounding box coordinates for each detection with the flag '-ext_output'. This external outputs flag will give you a few extra details about each detection within an image.

In [ ]:
# darknet run with external output flag to print bounding box coordinates
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg -ext_output
imShow('predictions.jpg')


## **9/ Train a YOLO model on your own data (MUNICH dataset)**

We can train any YOLO models on our own data by using the following command line (https://github.com/AlexeyAB/darknet)

```
 !./darknet detector train [data config file] [model config file] [pretrain-model weights] -dont_show > [file log saved]
 ```

 You can see that we need to create a data configuration file *(.data)* and a model configuration file *(.cfg)*. We also need a *'.names'* file which contains the names of objects to be detected.

 The necessary steps are described in the section: **How to train (to detect your custom objects)** of the repository.

 Let's practice with the MUNICH dataset downloaded from here:
 (https://filesender.renater.fr/?s=download&token=a3b67151-81b2-4bfa-bd9b-cce166daa833)

In [ ]:
!pwd

## **10/ Evaluate and compare different YOLO models on the VEDAI and XVIEW datasets**

We can train any YOLO models on our own data by using the following command line (https://github.com/AlexeyAB/darknet)

```
 !./darknet detector map [data config file] [model config file] [pretrain-model weights] -iou_thresh 0.xx -dont_show > [file log saved]
 ```

Data and configuration files (and pre-trained weights in case of not enough time) will be provided. You will be asked to test different models with different thresholds and create a comparison tables on different metrics.

